### HIGHWAY NET

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# highway_net

Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install keras_preprocessing
!pip install keras
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Activation, Input
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json
from keras import backend as K

In [ ]:
# Function to create DataFrame containing image paths and labels
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

# Function to extract features from images
def extract_features(images):
    features = []
    for image in images:
        img = load_img(image, grayscale=True, target_size=(48, 48))  # Resizing images to a consistent size
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

# Define directories for train and test data
TRAIN_DIR = 'drive/MyDrive/images/train'
TEST_DIR = 'drive/MyDrive/images/test'


In [ ]:
# Create DataFrames for train and test data
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

In [ ]:

test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

In [ ]:

# Extract features for train and test data
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

# Normalize features
x_train = train_features / 255.0
x_test = test_features / 255.0

# Encode labels
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

# Define Highway Network model architecture
def highwaynet(input_shape, num_units=64, num_layers=8, activation='relu', gate_bias=-2):
    input_layer = Input(shape=input_shape)
    previous_layer = input_layer
    for i in range(num_layers):
        dense = Dense(units=num_units, activation=activation)(previous_layer)
        gate = Dense(units=num_units, activation='sigmoid', bias_initializer=Constant(gate_bias))(previous_layer)
        output = Lambda(lambda x: x[0] * x[2] + x[1] * (1 - x[2]))([dense, previous_layer, gate])
        previous_layer = output
    return Model(inputs=input_layer, outputs=output)


In [ ]:

# Define input shape
input_shape = (48, 48, 1)

# Build Highway Network model with modified hyperparameters
model = highwaynet(input_shape, num_units=128, num_layers=10, activation='relu', gate_bias=-3)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x=x_train, y=y_train, batch_size=128, epochs=80, validation_data=(x_test, y_test))

# Save model to disk
model.save("emotiondetector_latest_highwaynet_v2.h5")

# Load model from disk
model = model_from_json(model.to_json())
model.load_weights("emotiondetector_latest_highwaynet_v2.h5")

pickle.dump(model,open('/content/drive/MyDrive/comparison_model_model/trained_model_dense', 'wb'))


In [ ]:

# Define labels
label = ['angry', 'happy', 'neutral', 'sad', 'surprise']

# Function to extract features from image
def ef(image):
    img = load_img(image, grayscale=True, target_size=(48, 48))  # Resizing images to a consistent size
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

# Prediction and visualization
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)

# Display the image
plt.imshow(img.reshape(48, 48), cmap='gray')
